# 详细指南

### 数据读入

In [1]:
from fastNLP.io import CSVLoader

loader = CSVLoader(headers=('raw_sentence', 'label'), sep='\t')
dataset = loader.load("./sample_data/tutorial_sample_dataset.csv")
dataset[0]

{'raw_sentence': A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str}

Instance表示一个样本，由一个或多个field（域，属性，特征）组成，每个field有名字和值。

在初始化Instance时即可定义它包含的域，使用 "field_name=field_value"的写法。

In [2]:
from fastNLP import Instance

dataset.append(Instance(raw_sentence='fake data', label='0'))
dataset[-1]

{'raw_sentence': fake data type=str,
'label': 0 type=str}

### 数据处理

In [3]:
from fastNLP import Vocabulary

# 将所有字母转为小写, 并所有句子变成单词序列
dataset.apply(lambda x: x['raw_sentence'].lower(), new_field_name='sentence')
dataset.apply_field(lambda x: x.split(), field_name='sentence', new_field_name='words')

# 使用Vocabulary类统计单词，并将单词序列转化为数字序列
vocab = Vocabulary(min_freq=2).from_dataset(dataset, field_name='words')
vocab.index_dataset(dataset, field_name='words',new_field_name='words')

# 将label转为整数
dataset.apply(lambda x: int(x['label']), new_field_name='target')
dataset[0]

{'raw_sentence': A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str,
'sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'words': [4, 1, 6, 1, 1, 2, 1, 11, 153, 10, 28, 17, 2, 1, 10, 1, 28, 17, 2, 1, 5, 154, 6, 149, 1, 1, 23, 1, 6, 149, 1, 8, 30, 6, 4, 35, 3] type=list,
'target': 1 type=int}

In [4]:
# 增加长度信息
dataset.apply_field(lambda x: len(x), field_name='words', new_field_name='seq_len')
print(dataset[0])

{'raw_sentence': A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str,
'sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'words': [4, 1, 6, 1, 1, 2, 1, 11, 153, 10, 28, 17, 2, 1, 10, 1, 28, 17, 2, 1, 5, 154, 6, 149, 1, 1, 23, 1, 6, 149, 1, 8, 30, 6, 4, 35, 3] type=list,
'target': 1 type=int,
'seq_len': 37 type=int}


## 使用内置模块CNNText
设置为符合内置模块的名称

In [5]:
from fastNLP.models import CNNText

model_cnn = CNNText((len(vocab),50), num_classes=5, padding=2, dropout=0.1)
model_cnn

CNNText(
  (embed): Embedding(
    177, 50
    (dropout): Dropout(p=0.0)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(50, 3, kernel_size=(3,), stride=(1,), padding=(2,))
      (1): Conv1d(50, 4, kernel_size=(4,), stride=(1,), padding=(2,))
      (2): Conv1d(50, 5, kernel_size=(5,), stride=(1,), padding=(2,))
    )
  )
  (dropout): Dropout(p=0.1)
  (fc): Linear(in_features=12, out_features=5, bias=True)
)

我们在使用内置模块的时候，还应该使用应该注意把 field 设定成符合内置模型输入输出的名字。

In [6]:
from fastNLP import Const

dataset.rename_field('words', Const.INPUT)
dataset.rename_field('seq_len', Const.INPUT_LEN)
dataset.rename_field('target', Const.TARGET)

dataset.set_input(Const.INPUT, Const.INPUT_LEN)
dataset.set_target(Const.TARGET)

print(Const.INPUT)
print(Const.INPUT_LEN)
print(Const.TARGET)

words
seq_len
target


### 分割训练集/验证集/测试集

In [7]:
train_dev_data, test_data = dataset.split(0.1)
train_data, dev_data = train_dev_data.split(0.1)
len(train_data), len(dev_data), len(test_data)

(64, 7, 7)

### 训练(model_cnn)

### loss
训练模型需要提供一个损失函数

下面提供了一个在分类问题中常用的交叉熵损失。注意它的**初始化参数**。

pred参数对应的是模型的forward返回的dict的一个key的名字，这里是"output"。

target参数对应的是dataset作为标签的field的名字，这里是"label_seq"。

In [8]:
from fastNLP import CrossEntropyLoss

# loss = CrossEntropyLoss()
# 等价于
loss = CrossEntropyLoss(pred=Const.OUTPUT, target=Const.TARGET)

### Metric
定义评价指标

这里使用准确率。参数的“命名规则”跟上面类似。

pred参数对应的是模型的predict方法返回的dict的一个key的名字，这里是"predict"。

target参数对应的是dataset作为标签的field的名字，这里是"label_seq"。

In [9]:
from fastNLP import AccuracyMetric

# metrics=AccuracyMetric()
# 等价于
metrics=AccuracyMetric(pred=Const.OUTPUT, target=Const.TARGET)

In [10]:
from fastNLP import Trainer

trainer = Trainer(model=model_cnn, train_data=train_data, dev_data=dev_data, loss=loss, metrics=metrics)
trainer.train()

input fields after batch(if batch size is 2):
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 16]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2019-05-12-21-38-34


Evaluation at Epoch 1/10. Step:2/20. AccuracyMetric: acc=0.285714

Evaluation at Epoch 2/10. Step:4/20. AccuracyMetric: acc=0.428571

Evaluation at Epoch 3/10. Step:6/20. AccuracyMetric: acc=0.428571

Evaluation at Epoch 4/10. Step:8/20. AccuracyMetric: acc=0.428571

Evaluation at Epoch 5/10. Step:10/20. AccuracyMetric: acc=0.428571

Evaluation at Epoch 6/10. Step:12/20. AccuracyMetric: acc=0.428571

Evaluation at Epoch 7/10. Step:14/20. AccuracyMetric: acc=0.428571

Evaluation at Epoch 8/10. Step:16/20. AccuracyMetric: acc=0.857143

Evaluation at Epoch 9/10. Step:18/20. AccuracyMetric: acc=0.857143

Evaluation at Epoch 10/10. Step:20/20. AccuracyMetric: acc=0.857143


In Epoch:8/Step:16, got best dev performance:AccuracyMetric: acc=0.857143
Reloaded the best model.


{'best_eval': {'AccuracyMetric': {'acc': 0.857143}},
 'best_epoch': 8,
 'best_step': 16,
 'seconds': 0.21}

### 测试(model_cnn)

In [11]:
from fastNLP import Tester

tester = Tester(test_data, model_cnn, metrics=AccuracyMetric())
tester.test()

[tester] 
AccuracyMetric: acc=0.857143


{'AccuracyMetric': {'acc': 0.857143}}

## 编写自己的模型

完全支持 pytorch 的模型，与 pytorch 唯一不同的是返回结果是一个字典，字典中至少需要包含 "pred" 这个字段

In [12]:
import torch
import torch.nn as nn

class LSTMText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, hidden_dim=64, num_layers=2, dropout=0.5):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, words):
        # (input) words : (batch_size, seq_len)
        words = words.permute(1,0)
        # words : (seq_len, batch_size)

        embedded = self.dropout(self.embedding(words))
        # embedded : (seq_len, batch_size, embedding_dim)
        output, (hidden, cell) = self.lstm(embedded)
        # output: (seq_len, batch_size, hidden_dim * 2)
        # hidden: (num_layers * 2, batch_size, hidden_dim)
        # cell: (num_layers * 2, batch_size, hidden_dim)

        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(hidden)
        # hidden: (batch_size, hidden_dim * 2)

        pred = self.fc(hidden.squeeze(0))
        # result: (batch_size, output_dim)
        return {"pred":pred}

In [13]:
model_lstm = LSTMText(len(vocab),50,5)
trainer = Trainer(model=model_lstm, train_data=train_data, dev_data=dev_data, loss=loss, metrics=metrics)
trainer.train()

input fields after batch(if batch size is 2):
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 16]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2019-05-12-21-38-36


Evaluation at Epoch 1/10. Step:2/20. AccuracyMetric: acc=0.571429

Evaluation at Epoch 2/10. Step:4/20. AccuracyMetric: acc=0.571429

Evaluation at Epoch 3/10. Step:6/20. AccuracyMetric: acc=0.571429

Evaluation at Epoch 4/10. Step:8/20. AccuracyMetric: acc=0.571429

Evaluation at Epoch 5/10. Step:10/20. AccuracyMetric: acc=0.714286

Evaluation at Epoch 6/10. Step:12/20. AccuracyMetric: acc=0.857143

Evaluation at Epoch 7/10. Step:14/20. AccuracyMetric: acc=0.857143

Evaluation at Epoch 8/10. Step:16/20. AccuracyMetric: acc=0.857143

Evaluation at Epoch 9/10. Step:18/20. AccuracyMetric: acc=0.857143

Evaluation at Epoch 10/10. Step:20/20. AccuracyMetric: acc=0.857143


In Epoch:6/Step:12, got best dev performance:AccuracyMetric: acc=0.857143
Reloaded the best model.


{'best_eval': {'AccuracyMetric': {'acc': 0.857143}},
 'best_epoch': 6,
 'best_step': 12,
 'seconds': 2.15}

In [14]:
tester = Tester(test_data, model_lstm, metrics=AccuracyMetric())
tester.test()

[tester] 
AccuracyMetric: acc=0.857143


{'AccuracyMetric': {'acc': 0.857143}}

### 使用 Batch编写自己的训练过程

In [15]:
from fastNLP import BucketSampler
from fastNLP import Batch
import torch
import time

model = CNNText((len(vocab),50), num_classes=5, padding=2, dropout=0.1)

def train(epoch, data):
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    lossfunc = torch.nn.CrossEntropyLoss()
    batch_size = 32

    # 定义一个Batch，传入DataSet，规定batch_size和去batch的规则。
    # 顺序（Sequential），随机（Random），相似长度组成一个batch（Bucket）
    train_sampler = BucketSampler(batch_size=batch_size, seq_len_field_name='seq_len')
    train_batch = Batch(batch_size=batch_size, dataset=data, sampler=train_sampler)
    
    start_time = time.time()
    for i in range(epoch):
        loss_list = []
        for batch_x, batch_y in train_batch:
            optim.zero_grad()
            output = model(batch_x['words'])
            loss = lossfunc(output['pred'], batch_y['target'])
            loss.backward()
            optim.step()
            loss_list.append(loss.item())
        print('Epoch {:d} Avg Loss: {:.2f}'.format(i, sum(loss_list) / len(loss_list)),end=" ")
        print('{:d}ms'.format(round((time.time()-start_time)*1000)))
        loss_list.clear()
            
train(10, train_data)
tester = Tester(test_data, model, metrics=AccuracyMetric())
tester.test()

Epoch 0 Avg Loss: 3.11 18ms
Epoch 1 Avg Loss: 2.88 30ms
Epoch 2 Avg Loss: 2.69 42ms
Epoch 3 Avg Loss: 2.47 54ms
Epoch 4 Avg Loss: 2.38 67ms
Epoch 5 Avg Loss: 2.10 78ms
Epoch 6 Avg Loss: 2.06 91ms
Epoch 7 Avg Loss: 1.92 103ms
Epoch 8 Avg Loss: 1.91 114ms
Epoch 9 Avg Loss: 1.76 126ms
[tester] 
AccuracyMetric: acc=0.571429


{'AccuracyMetric': {'acc': 0.571429}}

### 使用 Callback 实现自己想要的效果

In [16]:
from fastNLP import Callback

start_time = time.time()

class MyCallback(Callback):
    def on_epoch_end(self):
        print('Sum Time: {:d}ms\n\n'.format(round((time.time()-start_time)*1000)))
        

model = CNNText((len(vocab),50), num_classes=5, padding=2, dropout=0.1)
trainer = Trainer(model=model, train_data=train_data, dev_data=dev_data,
                  loss=CrossEntropyLoss(), metrics=AccuracyMetric(), callbacks=[MyCallback()])
trainer.train()

input fields after batch(if batch size is 2):
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 16]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2019-05-12-21-38-40


Evaluation at Epoch 1/10. Step:2/20. AccuracyMetric: acc=0.285714

Sum Time: 51ms


Evaluation at Epoch 2/10. Step:4/20. AccuracyMetric: acc=0.285714

Sum Time: 69ms


Evaluation at Epoch 3/10. Step:6/20. AccuracyMetric: acc=0.285714

Sum Time: 91ms


Evaluation at Epoch 4/10. Step:8/20. AccuracyMetric: acc=0.571429

Sum Time: 107ms


Evaluation at Epoch 5/10. Step:10/20. AccuracyMetric: acc=0.571429

Sum Time: 125ms


Evaluation at Epoch 6/10. Step:12/20. AccuracyMetric: acc=0.571429

Sum Time: 142ms


Evaluation at Epoch 7/10. Step:14/20. AccuracyMetric: acc=0.571429

Sum Time: 158ms


Evaluation at Epoch 8/10. Step:16/20. AccuracyMetric: acc=0.571429

Sum Time: 176ms


Evaluation at Epoch 9/10. Step:18/20. AccuracyMetric: acc=0.714286

Sum Time: 193ms


Evaluation at Epoch 10/10. Step:20/20. AccuracyMetric: acc=0.857143

Sum Time: 212ms



In Epoch:10/Step:20, got best dev performance:AccuracyMetric: acc=0.857143
Reloaded the best model.


{'best_eval': {'AccuracyMetric': {'acc': 0.857143}},
 'best_epoch': 10,
 'best_step': 20,
 'seconds': 0.2}